In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Reshaping with Hierarchical Indexing

- stack : This “rotates” or pivots from the columns in the data to the rows
- unstack : This pivots from the rows into the columns

In [2]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                     index=pd.Index(['Ohio', 'Colorado'], name='state'),
                     columns=pd.Index(['one', 'two', 'three'],
                     name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [3]:
# Using the stack method on this data pivots the columns into the rows, producing a Series:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [4]:
# From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with unstack
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [5]:
# unstack a different level by passing a level number or name
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [6]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [7]:
# Unstacking might introduce missing data if all of the values in the level aren't found in each of subgroups:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [8]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [9]:
# stacking filters out missing data by default,
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [10]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [11]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [12]:
# When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result
df = pd.DataFrame({'left': result, 'right': result + 5},
                   columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [13]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [14]:
# when calling stack, we can indicate the name of the axis to stack
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## Pivoting “Long” to “Wide” Format

In [15]:
data = pd.read_csv('dataset/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [16]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                          name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [17]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [18]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
1960-06-30 23:59:59.999999999,0.14,2834.390,5.2
1960-09-30 23:59:59.999999999,2.70,2839.022,5.6
1960-12-31 23:59:59.999999999,1.21,2802.616,6.3
1961-03-31 23:59:59.999999999,-0.40,2819.264,6.8


In [19]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,1.738539
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.181547
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.074120
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.820111
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.057235
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.144334
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.758321
7,1959-09-30 23:59:59.999999999,infl,2.740,-2.123612
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.133513
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-3.466426


In [20]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.181547  1.738539   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.057235 -0.820111   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -2.123612  0.758321   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.047284 -3.466426   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.401100  0.276521   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.074120  
1959-06-30 23:59:59.999999999  0.144334  
1959-09-30 23:59:59.999999999  0.133513  
1959-12-31 23:59:59.999999999 -1.188718  
1960-03-31 23:59:59.999999999  0.649393

In [21]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [22]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.181547  1.738539   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.057235 -0.820111   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -2.123612  0.758321   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.047284 -3.466426   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.401100  0.276521   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.538848  0.179891   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  0.495586 -0.716913   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.074120  
1959-06-30 23:59:59.999999999  0.144334  
1959-09-30 23:59:59.999999999  0.133513  
1959-12-31 23:59:59.999999999 -1.188718  
1960-03-31 23:59:59.999999999  0.649393  
1960-06-30 23:59:59.999999999  1.768905  
1960-09-30 23:59:59.999999999  1.374348

## Pivoting “Wide” to “Long” Format

In [23]:
# an inverse operation to pivot for DataFrames is pandas.melt.
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                    'A': [1, 2, 3],
                    'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [24]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [25]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [26]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [27]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [28]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [29]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
